In [1]:
from tkinter.filedialog import askopenfilename
import PyPDF2, re, os
import pandas as pd

In [2]:
# Define the columns
columns = [
    'PACKED BY',
    'QTY ORDERED',
    'QTY SHIPPED',
    'ITEM NUMBER',
    'Description',
    'List Price',
    'Your Price',
    'List Extended',
    'Your Extended',
    'Discounted Amount'
]

In [57]:
# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    pdf_text = ""
    invoiceNo = ""
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()
            if invoiceNo == "":
                invoiceNo = re.search(r'300\d{7}', text).group()
            if page_num == len(reader.pages)-1:
                pdf_text += text[text.find('Discounted\nAmount')+18:max([text.find('BOX :'), text.find('BXS. :')])]
            else:
                pdf_text += text[text.find('Discounted\nAmount')+18:text.find('1. Returns')]
    return pdf_text, invoiceNo

In [63]:
# Function to parse the extracted text and extract invoice data
def parse_invoice_data(pdf_text):
    lines = pdf_text.split("\n")
    cursor = 0
    invoice_data = []
    row_data = []
    for line in lines:
        if cursor%len(columns) == 0 and row_data != []:
            invoice_data.append(row_data)
            row_data = []

        if cursor%len(columns) == 2 and not line.isnumeric():
            row_data[-1] += line
        elif cursor%len(columns) == 4:
            if '$' in line:
                row_data.append('0')
                cursor += 1
            row_data.append(line)
            cursor += 1
        else:
            row_data.append(line)
            cursor += 1
    return invoice_data

In [58]:
# file_path = 'test_data/3000286319-IVC(06262024).PDF' # 1 Page
# file_path = 'test_data/3000285850-IVC(06262024).PDF' # 2 Pages
file_path = 'test_data/3000280847-IVC(06172024).PDF' # 3 Pages


In [115]:
pdf_text, invoiceNo = extract_text_from_pdf(file_path)
# print(invoiceNo)
print(pdf_text)

YAI 40 40 SKBEX12 FB BEACH CURL 12" 5.25
3.25210.00
130.00 80.00
1-10 4-10 27- 10 30- 10
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
YAI 10 10 SKBEX12 FB BEACH CURL 12" 5.75
3.7557.50
37.50 20.00
OT27 -10   
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
YAI 180 180 SKBTX28 FB 3X BRAID 301 28" 5.00
3.00900.00
540.00 360.00
1-60 1B-120   
 _ _ _ _ _ _ _ _ _ _ _ _ _ 

In [66]:
test = pdf_text.split('\n')

In [114]:
print(test[1])
print(re.findall('\d+\.\d{2}',test[1]))

3.25210.00
['3.25', '210.00']


In [ ]:
invoice_data = parse_invoice_data(pdf_text)

In [ ]:
df = pd.DataFrame(invoice_data, columns=columns)
df['Item Code'] = df['Item Code'].str.slice(stop=18)